In [1]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from scipy import sparse
from datetime import datetime
from catboost import CatBoostRegressor

C:\Users\Evgeny\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
movies_imdb = pd.read_csv('./movies_imdb.csv')

In [3]:
als_train_u0_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_0.csv')
als_train_u0_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_1.csv')
als_train_u0_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_0_m_2.csv')
als_train_u1_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_0.csv')
als_train_u1_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_1.csv')
als_train_u1_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_1_m_2.csv')
als_train_u2_m0 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_0.csv')
als_train_u2_m1 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_1.csv')
als_train_u2_m2 = pd.read_csv('./ALS/ratings_train_with_all_ALS_predictions_u_2_m_2.csv')

als_test = pd.read_csv('./ALS/ratings_dev_with_all_ALS_predictions.csv')
als_final_test = pd.read_csv('./ALS/ratings_test_with_all_ALS_predictions.csv')

In [4]:
a = pd.read_csv('./predictions_linear/predictions_0', header=None)

In [5]:
print(a.shape)
print(als_train_u0_m0.shape)
print(a.head(3))
print(als_train_u0_m0.head(3))

(1567557, 5)
(1567557, 7)
   0       1   2          3         4
0  0   53434  19  822873600  2.558093
1  0  124035  47  823185196  3.624575
2  0  124035  52  823185197  3.384482
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   
2    33794  113116  1297394692     3.0        3.999297        3.608127   

   user_actor_ALS  
0        3.240306  
1        3.690494  
2        3.915428  


In [6]:
lin_features_0 = pd.read_csv('./predictions_linear/predictions_0', header=None)
lin_features_1 = pd.read_csv('./predictions_linear/predictions_1', header=None)
lin_features_2 = pd.read_csv('./predictions_linear/predictions_2', header=None)
lin_features_3 = pd.read_csv('./predictions_linear/predictions_3', header=None)
lin_features_4 = pd.read_csv('./predictions_linear/predictions_4', header=None)
lin_features_5 = pd.read_csv('./predictions_linear/predictions_5', header=None)
lin_features_6 = pd.read_csv('./predictions_linear/predictions_6', header=None)
lin_features_7 = pd.read_csv('./predictions_linear/predictions_7', header=None)
lin_features_8 = pd.read_csv('./predictions_linear/predictions_8', header=None)

In [7]:
lin_features_dev = pd.read_csv('./predictions_linear/dev_predictions', header=None)
lin_features_final_test = pd.read_csv('./predictions_linear/test_predictions', header=None)

In [8]:
lin_features_0.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_1.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_2.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_3.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_4.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_5.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_6.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_7.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_8.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)

In [9]:
lin_features_dev.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)
lin_features_final_test.rename(columns={0: 'fold', 1: 'userId', 2: 'movieId', 3: 'timestamp', 4: 'lin_pred'}, inplace=True)

In [10]:
print(lin_features_1.shape)
print(als_train_u0_m1.shape)

(1544890, 5)
(1544890, 7)


In [11]:
lin_features_0.head(2)

,fold,userId,movieId,timestamp,lin_pred
0,0,53434,19,822873600,2.558093
1,0,124035,47,823185196,3.624575


In [12]:
als_train_u0_m0 = pd.merge(als_train_u0_m0, lin_features_0, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m1 = pd.merge(als_train_u0_m1, lin_features_1, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u0_m2 = pd.merge(als_train_u0_m2, lin_features_2, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m0 = pd.merge(als_train_u1_m0, lin_features_3, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m1 = pd.merge(als_train_u1_m1, lin_features_4, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u1_m2 = pd.merge(als_train_u1_m2, lin_features_5, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m0 = pd.merge(als_train_u2_m0, lin_features_6, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m1 = pd.merge(als_train_u2_m1, lin_features_7, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])
als_train_u2_m2 = pd.merge(als_train_u2_m2, lin_features_8, how='left', 
                           left_on=['userId', 'movieId', 'timestamp'], 
                           right_on=['userId', 'movieId', 'timestamp'])

In [13]:
als_test = pd.merge(als_test, lin_features_dev, how='left', 
                    left_on=['userId', 'movieId', 'timestamp'], 
                    right_on=['userId', 'movieId', 'timestamp'])
als_final_test = pd.merge(als_final_test, lin_features_final_test, how='left', 
                          left_on=['userId', 'movieId', 'timestamp'], 
                          right_on=['userId', 'movieId', 'timestamp'])

In [14]:
print(lin_features_dev.shape)
print(als_test.shape)

(1702634, 5)
(1690360, 9)


In [15]:
print(lin_features_final_test.shape)
print(als_final_test.shape)

(1690360, 5)
(3790860, 8)


In [16]:
als_train_u1_m1.head(2)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred
0,6502,10416,1127117051,4.0,3.706461,3.368054,NaN,4,3.625825
1,24,23457,1205769764,5.0,3.173135,3.481067,3.033822,4,3.106729


In [17]:
als_test.head(2)

,userId,movieId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred
0,40965,1217,1180039623,5.0,3.616524,3.165045,3.187873,NaN,NaN
1,131172,466,835337608,4.0,2.412944,3.676014,2.545692,NaN,NaN


In [18]:
als_train = pd.concat([als_train_u0_m0, als_train_u0_m1, als_train_u0_m2,
                       als_train_u1_m0, als_train_u1_m1, als_train_u1_m2,
                       als_train_u2_m0, als_train_u2_m1, als_train_u2_m2],
                      ignore_index=True)

In [19]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,movieId,title,genres,titleCleaned,year,nconst,averageRating,numVotes
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",3212,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,5.5,1432.0
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",74859,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,6.8,2764.0
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,3167,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,7.0,10324.0


In [20]:
print(movies_imdb.shape)
print(movies_imdb['movieId'].nunique())

(19636, 18)
19636


In [21]:
movies_imdb['averageRating'] = movies_imdb['averageRating'] / 2

In [22]:
genres = list(map(lambda x: [el for el in x.split(',')] + ['None'] * (3 - len(x.split(','))), movies_imdb['genres_imdb']))

In [23]:
movies_imdb['genres_imdb_0'] = [el[0] for el in genres]
movies_imdb['genres_imdb_1'] = [el[1] for el in genres]
movies_imdb['genres_imdb_2'] = [el[2] for el in genres]

In [24]:
sum(movies_imdb['year'].isna())

0

In [25]:
movies_imdb['runtimeMinutes'][movies_imdb['runtimeMinutes'] == '\\N'] = -1

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
movies_imdb.head(3)

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_imdb,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,0,tt0066856,movie,born to win,born to win,0,1971,\N,88,"Comedy,Crime,Drama",...,Born to Win (1971),Drama,born to win,1971,nm0005837nm0001719nm0696038nm0000947nm0282031n...,2.75,1432.0,Comedy,Crime,Drama
1,1,tt0066858,movie,boy friend,boy friend,0,1971,\N,137,"Comedy,Musical,Romance",...,"Boy Friend, The (1971)",Comedy|Musical|Romance,boy friend,1971,nm0910589nm0493341nm0300083nm0012451nm0697799n...,3.40,2764.0,Comedy,Musical,Romance
2,2,tt0066892,movie,crnl knowledge,crnl knowledge,0,1971,\N,98,Drama,...,Carnal Knowledge (1971),Comedy|Drama,crnl knowledge,1971,nm0000197nm0000298nm0000268nm0307316nm0001566n...,3.50,10324.0,Drama,None,None


In [27]:
print(als_train.shape)
print(als_test.shape)
print(als_train.head(2))
print(als_test.head(2))
print(als_final_test.shape)
print(als_final_test.head(2))

(14453730, 9)
(1690360, 9)
   movieId  userId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0    33794  108543  1141315698     4.0        4.061448        3.435761   
1    33794  116656  1308349074     4.5        4.055828        3.234410   

   user_actor_ALS  fold  lin_pred  
0        3.240306     0  3.770295  
1        3.690494     0  3.869308  
   userId  movieId   timestamp  rating  user_movie_ALS  user_genre_ALS  \
0   40965     1217  1180039623     5.0        3.616524        3.165045   
1  131172      466   835337608     4.0        2.412944        3.676014   

   user_actor_ALS  fold  lin_pred  
0        3.187873   NaN       NaN  
1        2.545692   NaN       NaN  
(3790860, 8)
   userId  movieId   timestamp  user_movie_ALS  user_genre_ALS  \
0    2711     5902  1044313557        4.242603        3.521633   
1    3352     1639   895849643        3.059602        3.224138   

   user_actor_ALS  fold  lin_pred  
0             NaN   NaN       NaN  
1        3.300966   NaN    

In [28]:
def rmse_asymmetric(rating, prediction):
    bad_border = 4
    
    if rating < bad_border <= prediction:
        penalty_coefficient = 2.0
    else:
        penalty_coefficient = 1.0
    
    return penalty_coefficient*(rating - prediction)**2


def vector_rmse_asymmetric(df, true_col, pred_col, bad_border=4):
    data = np.array(df[[true_col, pred_col]].values)
    mask_1 = data[:, 0] < bad_border
    mask_2 = data[:, 1] >= bad_border
    penalty = data[mask_1 & mask_2]
    not_penalty = data[~mask_1 & mask_2]
    result = np.mean(np.concatenate((2 * (penalty[:, 0] - penalty[:, 1])  ** 2,
                                    (not_penalty[:, 0] - not_penalty[:, 1])  ** 2), 
                                    axis=None))
    return result

In [29]:
als_train = pd.merge(als_train, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_test = pd.merge(als_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])
als_final_test = pd.merge(als_final_test, movies_imdb, how='left', 
                     left_on=['movieId'], right_on=['movieId'])

In [30]:
als_train['titleType'].fillna('None', inplace=True)
als_test['titleType'].fillna('None', inplace=True)
als_final_test['titleType'].fillna('None', inplace=True)

als_train['genres_imdb_0'].fillna('None', inplace=True)
als_test['genres_imdb_0'].fillna('None', inplace=True)
als_final_test['genres_imdb_0'].fillna('None', inplace=True)

als_train['genres_imdb_1'].fillna('None', inplace=True)
als_test['genres_imdb_1'].fillna('None', inplace=True)
als_final_test['genres_imdb_1'].fillna('None', inplace=True)

als_train['genres_imdb_2'].fillna('None', inplace=True)
als_test['genres_imdb_2'].fillna('None', inplace=True)
als_final_test['genres_imdb_2'].fillna('None', inplace=True)

In [31]:
als_train.head(3)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,title,genres,titleCleaned,year,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
1,33794,116656,1308349074,4.5,4.055828,3.234410,3.690494,0,3.869308,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None
2,33794,113116,1297394692,3.0,3.999297,3.608127,3.915428,0,3.770065,9943.0,...,Batman Begins (2005),Action|Crime|IMAX,btmn begins,2005.0,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None


In [32]:
als_test.columns

Index(['userId', 'movieId', 'timestamp', 'rating', 'user_movie_ALS',
       'user_genre_ALS', 'user_actor_ALS', 'fold', 'lin_pred', 'Unnamed: 0',
       'tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres_imdb', 'title',
       'genres', 'titleCleaned', 'year', 'nconst', 'averageRating', 'numVotes',
       'genres_imdb_0', 'genres_imdb_1', 'genres_imdb_2'],
      dtype='object')

In [33]:
als_train['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_train['timestamp']))
als_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_test['timestamp']))
als_final_test['curr_date'] = list(map(lambda x: datetime.fromtimestamp(x).isoformat(), als_final_test['timestamp']))

In [34]:
als_train['curr_year'] = list(map(lambda x: float(x[:4]), als_train['curr_date']))
als_test['curr_year'] = list(map(lambda x: float(x[:4]), als_test['curr_date']))
als_final_test['curr_year'] = list(map(lambda x: float(x[:4]), als_final_test['curr_date']))

als_train['curr_month'] = list(map(lambda x: x[5:7], als_train['curr_date']))
als_test['curr_month'] = list(map(lambda x: x[5:7], als_test['curr_date']))
als_final_test['curr_month'] = list(map(lambda x: x[5:7], als_final_test['curr_date']))

als_train['movie_age'] = als_train['curr_year'] - als_train['year']
als_test['movie_age'] = als_test['curr_year'] - als_test['year']
als_final_test['movie_age'] = als_final_test['curr_year'] - als_final_test['year']

In [35]:
als_train.head(1)

,movieId,userId,timestamp,rating,user_movie_ALS,user_genre_ALS,user_actor_ALS,fold,lin_pred,Unnamed: 0,...,nconst,averageRating,numVotes,genres_imdb_0,genres_imdb_1,genres_imdb_2,curr_date,curr_year,curr_month,movie_age
0,33794,108543,1141315698,4.0,4.061448,3.435761,3.240306,0,3.770295,9943.0,...,nm0746273nm0000288nm0000323nm0913822nm0000553n...,4.1,1224005.0,Action,Adventure,None,2006-03-02T19:08:18,2006.0,03,1.0


In [36]:
print(vector_rmse_asymmetric(als_train, 'rating', 'user_movie_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_genre_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'user_actor_ALS'))
print(vector_rmse_asymmetric(als_train, 'rating', 'averageRating'))
print(vector_rmse_asymmetric(als_train, 'rating', 'lin_pred'))

C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


0.6197099090875856
1.0423400236647804
0.7162128806061265
1.3887644861972723
1.4576650118188108


In [37]:
feature_columns = [
    'user_movie_ALS', 
    'user_genre_ALS', 
    'user_actor_ALS', 
    'averageRating', 
    'isAdult', 
    'runtimeMinutes', 
    'numVotes', 
    #'year', 
    'movie_age', 
    #'curr_year', 
    #'lin_pred'
]
cat_feature_columns = [
    #'titleType', 
    #'genres_imdb_0', 
    #'genres_imdb_1', 
    #'genres_imdb_2', 
    #'curr_month'
]
train_columns = feature_columns + cat_feature_columns

In [38]:
train_data = als_train[train_columns]
train_target = als_train['rating']

test_data = als_test[train_columns]
test_target = als_test['rating']

In [39]:
bad_border = 4
mask_1 = np.array(train_target.values) < bad_border
weights = np.ones(len(train_data))
weights[mask_1] = 2

In [40]:
weights

array([1., 1., 2., ..., 2., 1., 2.])

In [41]:
model = CatBoostRegressor(iterations=500)

In [42]:
model.fit(train_data, train_target, verbose=True, sample_weight=weights, cat_features=cat_feature_columns)

0:	learn: 3.3413932	total: 1.12s	remaining: 9m 21s
1:	learn: 3.2472949	total: 2.16s	remaining: 8m 58s
2:	learn: 3.1560980	total: 3.19s	remaining: 8m 48s
3:	learn: 3.0678892	total: 4.24s	remaining: 8m 45s
4:	learn: 2.9824844	total: 5.28s	remaining: 8m 42s
5:	learn: 2.8998389	total: 6.56s	remaining: 9m
6:	learn: 2.8197642	total: 7.87s	remaining: 9m 14s
7:	learn: 2.7422953	total: 9.05s	remaining: 9m 16s
8:	learn: 2.6673069	total: 10.2s	remaining: 9m 15s
9:	learn: 2.5949555	total: 11.4s	remaining: 9m 18s
10:	learn: 2.5247602	total: 12.5s	remaining: 9m 16s
11:	learn: 2.4568909	total: 13.7s	remaining: 9m 15s
12:	learn: 2.3912744	total: 14.8s	remaining: 9m 15s
13:	learn: 2.3277781	total: 16s	remaining: 9m 15s
14:	learn: 2.2665120	total: 17.2s	remaining: 9m 16s
15:	learn: 2.2072210	total: 18.4s	remaining: 9m 15s
16:	learn: 2.1499759	total: 19.5s	remaining: 9m 14s
17:	learn: 2.0946725	total: 20.7s	remaining: 9m 14s
18:	learn: 2.0412872	total: 21.9s	remaining: 9m 14s
19:	learn: 1.9897236	total: 

156:	learn: 0.7963971	total: 3m 5s	remaining: 6m 44s
157:	learn: 0.7963549	total: 3m 6s	remaining: 6m 43s
158:	learn: 0.7963173	total: 3m 7s	remaining: 6m 42s
159:	learn: 0.7962797	total: 3m 8s	remaining: 6m 40s
160:	learn: 0.7962453	total: 3m 9s	remaining: 6m 39s
161:	learn: 0.7962123	total: 3m 11s	remaining: 6m 38s
162:	learn: 0.7961774	total: 3m 12s	remaining: 6m 38s
163:	learn: 0.7961448	total: 3m 13s	remaining: 6m 36s
164:	learn: 0.7961165	total: 3m 14s	remaining: 6m 35s
165:	learn: 0.7960872	total: 3m 15s	remaining: 6m 34s
166:	learn: 0.7960601	total: 3m 17s	remaining: 6m 33s
167:	learn: 0.7960313	total: 3m 18s	remaining: 6m 32s
168:	learn: 0.7960049	total: 3m 19s	remaining: 6m 31s
169:	learn: 0.7959827	total: 3m 20s	remaining: 6m 30s
170:	learn: 0.7959568	total: 3m 22s	remaining: 6m 28s
171:	learn: 0.7959361	total: 3m 23s	remaining: 6m 27s
172:	learn: 0.7959142	total: 3m 24s	remaining: 6m 26s
173:	learn: 0.7958956	total: 3m 25s	remaining: 6m 25s
174:	learn: 0.7958770	total: 3m 2

309:	learn: 0.7949585	total: 6m 3s	remaining: 3m 42s
310:	learn: 0.7949551	total: 6m 4s	remaining: 3m 41s
311:	learn: 0.7949502	total: 6m 5s	remaining: 3m 40s
312:	learn: 0.7949470	total: 6m 6s	remaining: 3m 38s
313:	learn: 0.7949435	total: 6m 7s	remaining: 3m 37s
314:	learn: 0.7949395	total: 6m 8s	remaining: 3m 36s
315:	learn: 0.7949353	total: 6m 9s	remaining: 3m 35s
316:	learn: 0.7949315	total: 6m 10s	remaining: 3m 34s
317:	learn: 0.7949285	total: 6m 12s	remaining: 3m 32s
318:	learn: 0.7949252	total: 6m 13s	remaining: 3m 31s
319:	learn: 0.7949211	total: 6m 14s	remaining: 3m 30s
320:	learn: 0.7949169	total: 6m 15s	remaining: 3m 29s
321:	learn: 0.7949132	total: 6m 16s	remaining: 3m 28s
322:	learn: 0.7949118	total: 6m 17s	remaining: 3m 27s
323:	learn: 0.7949080	total: 6m 18s	remaining: 3m 25s
324:	learn: 0.7949058	total: 6m 20s	remaining: 3m 24s
325:	learn: 0.7949029	total: 6m 21s	remaining: 3m 23s
326:	learn: 0.7948985	total: 6m 22s	remaining: 3m 22s
327:	learn: 0.7948947	total: 6m 23s

463:	learn: 0.7945158	total: 9m 8s	remaining: 42.5s
464:	learn: 0.7945134	total: 9m 9s	remaining: 41.3s
465:	learn: 0.7945112	total: 9m 10s	remaining: 40.2s
466:	learn: 0.7945097	total: 9m 11s	remaining: 39s
467:	learn: 0.7945071	total: 9m 12s	remaining: 37.8s
468:	learn: 0.7945050	total: 9m 13s	remaining: 36.6s
469:	learn: 0.7945031	total: 9m 15s	remaining: 35.4s
470:	learn: 0.7945011	total: 9m 16s	remaining: 34.2s
471:	learn: 0.7944994	total: 9m 17s	remaining: 33.1s
472:	learn: 0.7944973	total: 9m 18s	remaining: 31.9s
473:	learn: 0.7944953	total: 9m 19s	remaining: 30.7s
474:	learn: 0.7944931	total: 9m 20s	remaining: 29.5s
475:	learn: 0.7944914	total: 9m 22s	remaining: 28.3s
476:	learn: 0.7944889	total: 9m 23s	remaining: 27.2s
477:	learn: 0.7944870	total: 9m 24s	remaining: 26s
478:	learn: 0.7944848	total: 9m 25s	remaining: 24.8s
479:	learn: 0.7944838	total: 9m 26s	remaining: 23.6s
480:	learn: 0.7944827	total: 9m 27s	remaining: 22.4s
481:	learn: 0.7944805	total: 9m 28s	remaining: 21.2s

In [43]:
catboost_predict = model.predict(test_data)

In [44]:
als_test['catboost_predict'] = catboost_predict

In [45]:
print(vector_rmse_asymmetric(als_test, 'rating', 'catboost_predict'))

0.6164227880536982


In [46]:
model.feature_importances_

array([9.56751815e+01, 1.26458764e+00, 7.26530242e-01, 6.10963869e-01,
       2.91613439e-02, 1.79645481e-01, 8.40737713e-01, 6.73192238e-01])

In [47]:
als_test['mean_predict'] = np.mean(test_data[['user_movie_ALS', 'user_genre_ALS', 'user_actor_ALS', 'averageRating']].values, axis=1)

In [48]:
print(vector_rmse_asymmetric(als_test, 'rating', 'mean_predict'))

0.6475399516024992


C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


In [49]:
als_test['median_predict'] = np.median(test_data[['user_movie_ALS', 'user_genre_ALS', 'user_actor_ALS', 'averageRating']].values, axis=1)

C:\Users\Evgeny\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median for 671110 results
  r = func(a, **kwargs)


In [50]:
print(vector_rmse_asymmetric(als_test, 'rating', 'median_predict'))

0.6499199236058524


C:\Users\Evgeny\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


In [51]:
final_test_results = model.predict(als_final_test[train_columns])

In [52]:
als_final_test['prediction'] = final_test_results

In [ ]:
als_final_test.to_csv('./prediction_2.csv')

In [ ]:
import pickle

In [ ]:
with open('./saved_catboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)